In [69]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import warnings
warnings.filterwarnings("ignore")

In [70]:
df = pd.read_csv("titanic.csv")
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [87]:
df.fare.dtype


dtype('float64')

In [71]:
X = df.drop("survived", axis=1)
y = df.survived

In [72]:
class PreProcer(BaseEstimator, TransformerMixin):
    def fit(self,X, y=None):
        self.ageimpute = SimpleImputer()
        self.ageimpute.fit(X[["age"]])
        return self
    def transform(self,X, y=None):
        X["age"] = self.ageimpute.transform(X[["age"]])
        X["cabinClass"] = X["cabin"].fillna("M").apply(lambda x:str(x).replace(" ","")).apply(lambda x: re.sub(r"[^a-zA-Z]", '',x))
        X["cabinNumber"] = X["cabin"].fillna("M").apply(lambda x:str(x).replace(" ","")).apply(lambda x: re.sub(r"[^0-9]", '',x)).replace("",0)
        X["embarked"] = X["embarked"].fillna("M")
        X = X.drop(["name", "ticket", "cabin", "boat", "body","home.dest"], axis=1, errors="ignore")
        return X

In [73]:
preprocer = PreProcer()
numeric_pipe = Pipeline([("Scaler", StandardScaler())])
cat_pipe = Pipeline([("OneHot", OneHotEncoder(handle_unknown="ignore"))])
transform = ColumnTransformer([("num", numeric_pipe, ["pclass", "age", "sibsp", "parch", "fare", "cabinNumber"]),("cat", cat_pipe, ["sex", "embarked", "cabinClass"])])

In [74]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1234)

In [75]:
X.shape

(1309, 13)

In [76]:
x_train

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
588,2,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0000,NaN,S,14,NaN,"Cornwall / Akron, OH"
351,2,"Brown, Mr. Thomas William Solomon",male,60.0,1,1,29750,39.0000,NaN,S,NaN,NaN,"Cape Town, South Africa / Seattle, WA"
99,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,11755,39.6000,A16,C,1,NaN,London / Paris
398,2,"Drew, Master. Marshall Brines",male,8.0,0,2,28220,32.5000,NaN,S,10,NaN,"Greenport, NY"
522,2,"Otter, Mr. Richard",male,39.0,0,0,28213,13.0000,NaN,S,NaN,NaN,"Middleburg Heights, OH"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,3,"Barah, Mr. Hanna Assi",male,20.0,0,0,2663,7.2292,NaN,C,15,NaN,NaN
1228,3,"Stranden, Mr. Juho",male,31.0,0,0,STON/O 2. 3101288,7.9250,NaN,S,9,NaN,NaN
1077,3,"O'Driscoll, Miss. Bridget",female,NaN,0,0,14311,7.7500,NaN,Q,D,NaN,NaN
723,3,"Conlon, Mr. Thomas Henry",male,31.0,0,0,21332,7.7333,NaN,Q,NaN,NaN,"Philadelphia, PA"


In [77]:
mlp = Pipeline(
    [
        ("InitialPreProcer", PreProcer()),
        ("Transformer", transform),
        ("xgb", XGBClassifier())
    ]
    
)

In [78]:
mlp.fit(x_train,y_train)

Pipeline(steps=[('InitialPreProcer', PreProcer()),
                ('Transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'sibsp',
                                                   'parch', 'fare',
                                                   'cabinNumber']),
                                                 ('cat',
                                                  Pipeline(steps=[('OneHot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'embarked',
                                                   'cabinClass'])])),
                ('xgb',
                 XGBClassifier(base_score=Non...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [79]:
y_hat = mlp.predict(x_test)

In [80]:
precision_score(y_test,y_hat)

0.7857142857142857

In [81]:
import joblib

In [82]:
joblib.dump(mlp, "xgbpipe.joblib")

['xgbpipe.joblib']